# Uebung 1

## Aufgabe 1

Der Datensatz Bauprojekte Infrastruktur der SBB (https://opendata.swiss/de/dataset/bauprojekte-infrastruktur) enhält 

a) Was muss bei der Lizenz berücksichtigt werden ?

b) Stellen Sie alle aktuellen Bauwerke mit Markern auf einer Karte mit Folium und Markern (optional Markercluster) dar.



In [ ]:
import folium
import urllib
import shutil

In [ ]:
# CSV herunterladen:
filename, headers = urllib.request.urlretrieve("https://data.sbb.ch/api/v2/catalog/datasets/construction-projects/exports/csv", "daten/bauprojekte.csv")

In [ ]:
m = folium.Map(location=[47.37825,8.5367835], tiles='', zoom_start=7)
 
folium.raster_layers.TileLayer(
    tiles="https://wmts.geo.admin.ch/1.0.0/ch.swisstopo.pixelkarte-farbe/default/current/3857/{z}/{x}/{y}.jpeg",
    attr="(c) swisstopo",
    name="pixelkarte-farbe",
    min_zoom=8,
    max_zoom=18,
    tms=False,
    overlay=False,
    control=False,
).add_to(m)

folium.LayerControl().add_to(m)


file = open("daten/bauprojekte.csv", encoding="utf-8")

next(file)

for line in file:
    data = line.strip().split(";")
    lnglat = data[0].split(",")
    lat = float(lnglat[0])
    lng = float(lnglat[1])
    projektname = data[1]
    ort = data[2]
    link = data[5]
    url = ""
    if link != "":
        url = f"<a href='{link}' target='_blank'>link</a>"
    
    folium.Marker([lat,lng], 
              popup=f"{projektname}<br/>{ort}<br/>{url}",
              icon=folium.Icon(color="red", prefix="fa", icon="star")).add_to(m)
   

file.close()
    
m


## Aufgabe 2

In [ ]:
# CSV herunterladen:
filename, headers = urllib.request.urlretrieve("https://data.sbb.ch/api/v2/catalog/datasets/construction-projects/exports/geojson", "daten/bauprojekte.geojson")

In [ ]:
import geopandas as gpd
import pandas as pd

cantons = gpd.read_file("daten/gemeindegrenzen/ggg_2021-LV95/shp/g1k21.shp", encoding="utf-8")
points = gpd.read_file("daten/bauprojekte.geojson", encoding="utf-8")

cantons.head()

points = points.to_crs(2056)

# Spatial Join
pointsInPolygon = gpd.sjoin(points, cantons, how="inner", op='intersects')


In [ ]:
pointsInPolygon['anzahl'] = 1
pointsInPolygon = pointsInPolygon[["KTNR", "anzahl"]]

result = pointsInPolygon.groupby(['KTNR']).sum()
result

## Aufgabe 3

Laden sie ein einzelnes Mosaik von swissimage herunter. Dieses kann mit dem Web-Interface auf https://www.swisstopo.admin.ch/de/geodata/images/ortho/swissimage10.html gefunden werden.
(durch klicken auf "alle links exportieren"). Speichern Sie dieses Bild im "daten" Ordner.

Beispiel:

```url = "https://data.geo.admin.ch/ch.swisstopo.swissimage-dop10/swissimage-dop10_2018_2615-1264/swissimage-dop10_2018_2615-1264_0.1_2056.tif"```


a) Bestimmen Sie unter verwendung von Rasterio:

    1. Die Auflösung des Bildes
    2. Wieviele Quadratmeter werden in diesem Bild abgedeckt?
    3. Die LV95 Koordinaten von den Eckpunkten und dem Mittelpunkt des Bildes
    
b) Plotten Sie das Bild in Jupyter Lab



In [ ]:
import rasterio
import geoutils
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
url = "https://data.geo.admin.ch/ch.swisstopo.swissimage-dop10/swissimage-dop10_2018_2615-1264/swissimage-dop10_2018_2615-1264_0.1_2056.tif"

In [ ]:
geoutils.download(url, "daten/swissimage_ausschnitt.tif")

In [ ]:
dataset = rasterio.open("daten/swissimage_ausschnitt.tif")

print(dataset.width, dataset.height)

In [ ]:
10000*10000*3/1024/1024

In [ ]:
dataset.crs

In [ ]:
dataset.transform

In [ ]:
s = dataset.transform[0]

In [ ]:
l = dataset.width * s
b = dataset.height * s
print(l,b)

In [ ]:
r = dataset.read(1)
g = dataset.read(2)
b = dataset.read(3)

rgb = np.dstack((r,g,b))

del r
del g
del b

In [ ]:
fig, ax = plt.subplots(figsize=(16,9))
ax.imshow(rgb, interpolation='bilinear');

In [ ]:
del rgb